In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC

In [ ]:
hz = 250 # частота оцифровки сигнала

In [ ]:
# Функции для работы с ЭКГ
def getPeaks(ecg, thold=0.3):
    """Вычисление индексов точек, соответствующих вершинам R-зубцов 
    thold - пороговое значение сигнала выше которого будут искаться пики"""
    i = 0
    peaks = []
    while i < len(ecg) - 1:
    if ecg[i] > 0.3 and ecg[i-1] < ecg[i] > ecg[i+1]:
        peaks.append(i)
        i += 100
    else:
        i += 1
    return np.array(peaks)

In [ ]:
def getIntervals(ecg):
    """Вычисление интервалов между R-зубцами"""
    peaks = getPeaks(ecg)
    intervals = np.zeros(len(peaks)-1)
    for i in range(len(intervals)):
        intervals[i] = peaks[i+1] - peaks[i]
    return intervals

In [ ]:
def getHR(ecg):
    """Вычисление ЧСС по индексам пиков. Вычисляется по крайним
    пикам в подаваемом в качестве аргумента сигнале"""
    peaks = getPeaks(ecg)
    hr = 60/((peaks[-1] - peaks[1])/hz/(len(peaks)-1))
    return hr

In [ ]:
# Функции для работы с КГР
def getDerivative(vector):
    """Вычисляет производную сигнала"""
    return vector[1:]-vector[:-1]

In [ ]:
def stabilize(vector, begin, end):
    """Убирает наклон графика КГР"""
    vector[end+1:] -= vector[end]-vector[begin]
    vector[begin:end+1] -= np.linspace(0, vector[end]-vector[begin], end-begin+1)

In [ ]:
def integrate(vector, const=0, thold=None):
    """Интегрирует сигнал. vector - массив производных некого сигнала,
    const - его начальное значение. Отсекает все производные, превышающие
    по модулю пороговое значение thold."""
    ans = np.zeros(len(vector)+1)
    ans[0] = const
    if thold is not None:
        absVec = np.abs(vector)
        if absVec[0] > thold:
            vector[0] = 0.0
        for i in range(1, len(vector)):
            if absVec[i] > thold:
                vector[i] = vector[i-1]
    for i, der in enumerate(vector):
        ans[i+1] = ans[i] + der
    return ans


In [ ]:
def getLocalMaxsGSR(gsr):
    """Возвращает индексы локальных максимумов сигнала КГР"""
    der = getDerivative(gsr)
    i = 10
    maxs = []
    while i < len(der) - 10:
        if der[i-10] < der[i] > der[i+10] and der[i]-der[i+10] > 1e-12\
                                        and der[i]-der[i-10] > 1e-12:
            maxs.append(i)
            i += 249
        i += 1
    maxs = np.array(maxs)
    return maxs

In [ ]:
def getClosestInterval(index, intervalsDict):
    """Возвращает ближайший к index временной интервал между
    локальными максимумами производной сигнала КГР"""
    closestIndex = min(intervalsDict.keys(), key=lambda x: abs(x-index))
    return intervalsDict[closestIndex]

In [ ]:
# Обработка ЭЭГ

def compl(sig, coef=0.95):
    """Комплементарный фильтр с коэффициентом coef.
    Возвращает сигнал после фильтрации."""
    for i in range(1, len(sig)):
        sig[i] = coef * sig[i-1] + (1-coef) * sig[i]
    return sig

In [ ]:
def med(sig, coef=15):
    """Возвращает массив длины len(sig)//coef, содержащий
    медианный значения отрезков sig длиной coef."""
    ret = np.zeros(len(sig)//coef)
    for i in range(len(ret)):
        ret[i] = np.average(sig[i*coef:(i+1)*coef])
    return ret

In [ ]:
def getAlBeta(eeg, chunkSize=15*hz):
    """Вычисляет уровни альфа- и бета-ритмов в сигнале.
    Вычисляется для chunkSize числа точек."""
    nChunks = eeg.shape[0]//chunkSize
    alphas = np.zeros(nChunks)
    betas = np.zeros(nChunks)
    coef = chunkSize // hz
    for i in range(nChunks):
        chunk = eeg[i*chunkSize:(i+1)*chunkSize]
        spec = np.fft.fft(chunk)
        spec = np.abs(spec)
        alphas[i] = sum(spec[8*coef:13*coef+1])
        betas[i] = sum(spec[15*coef:30*coef+1])
    return pd.DataFrame({'alpha': alphas, 'beta': betas})

In [ ]:
def getAlphaToBetaSmooth(eeg, coef=15):
    """Вычисляет отношение альфа-ритма к бета- в сигнале ЭЭГ.
    После получение уровней альфа- и бета-ритмов сглаживает их.
    Изначально уровни вычисляются во временном окне равном 1 с.
    Параметр coef определяет число последовательных значений
    уровней, по которым считается медианное значение."""
    new = getAlBeta(eeg, chunkSize=hz)
    alpha = new.alpha.values
    beta = new.beta.values
    alpha = compl(alpha)
    beta = compl(beta)
    alpha = med(alpha, coef)
    beta = med(beta, coef)
    alpha = compl(alpha)
    beta = compl(beta)
    alphaToBeta = compl(alpha/beta)
    
    return alphaToBeta